In [15]:
import pandas as pd

In [105]:
event=pd.read_csv("../data/demo_data/event/event.csv")
event.index=event["time"].astype("datetime64")
event=event[["event"]]
event

,event
time,
2021-07-10 00:00:00,cpu.idle load.average_cpu.5min
2021-07-10 00:01:00,cpu.idle load.average_cpu.5min
2021-07-10 00:02:00,cpu.idle load.average_cpu.5min
2021-07-10 00:03:00,cpu.idle load.average_cpu.5min
2021-07-10 00:04:00,cpu.idle load.average_cpu.5min
...,...
2021-07-10 03:54:00,load.1min cpu.idle
2021-07-10 03:55:00,cpu.idle load.average_cpu.5min
2021-07-10 03:56:00,cpu.idle load.average_cpu.5min


In [106]:
# 按照时间窗口聚类事件
groupby_event=event["event"].resample("15T").apply(lambda x:",".join(x))
groupby_event_df=pd.DataFrame()
groupby_event_df["events"]=groupby_event
groupby_event_df["itemset"]=groupby_event_df["events"].apply(lambda x:x.split(","))
train_data_set_with_duplic=groupby_event_df["itemset"].tolist()
# 去重
train_set=[]
for i in train_data_set_with_duplic:
    train_set.append(list(set(i)))
train_set

[['disk.io.util cpu.idle',
  'cpu.user sys.ntpd.offset',
  'disk.io.await disk.io.util',
  'cpu.idle sys.disk.rw',
  'cpu.idle load.average_cpu.5min',
  'load.5min disk.io.await cpu.idle',
  'load.5min cpu.user mem.memfree.percent_buffer_cache sys.ntpd.offset',
  'cpu.idle load.5min',
  'load.average_cpu.5min cpu.idle'],
 ['disk.io.util cpu.idle',
  'cpu.idle load.1min',
  'disk.io.await disk.io.util',
  'disk.io.util disk.io.await',
  'cpu.idle disk.io.util',
  'cpu.idle load.average_cpu.5min',
  'disk.io.util df.bytes.used.percent',
  'disk.io.util net.if.out.usage',
  'load.average_cpu.5min cpu.idle'],
 ['disk.io.util cpu.idle',
  'cpu.user sys.ntpd.offset',
  'disk.io.await disk.io.util',
  'net.if.out.usage df.bytes.used.percent',
  'mem.swapused.percent disk.io.util',
  'load.average_cpu.5min cpu.idle',
  'disk.io.util df.bytes.used.percent disk.io.await',
  'df.bytes.used.percent cpu.idle',
  'cpu.idle mem.swapused.percent disk.io.util',
  'df.bytes.used.percent disk.io.util',
 

In [93]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(train_set).transform(train_set)

df = pd.DataFrame(te_ary, columns=te.columns_)
df

,cpu.idle df.bytes.used.percent,cpu.idle disk.io.await disk.io.util,cpu.idle disk.io.util,cpu.idle disk.io.util df.bytes.used.percent,cpu.idle disk.io.util mem.swapused.percent,cpu.idle host.alive,cpu.idle load.1min,cpu.idle load.5min,cpu.idle load.average_cpu.5min,cpu.idle mem.swapused.percent,...,mem.memused.percent_buffer_cache mem.swapused.percent,mem.swapused.percent disk.io.util,mem.swapused.percent disk.io.util cpu.idle,mem.swapused.percent mem.memused.percent_buffer_cache,net.if.out.usage df.bytes.used.percent,net.if.out.usage disk.io.util,net.if.out.usage load.5min df.bytes.used.percent,net.if.out.usage net.if.in.usage load.5min,ngx.metric.count.error_50x ngx.metric.count.error_500,sys.disk.rw disk.io.util
0,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,True,False,False,False,False,False
3,False,False,True,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
6,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
8,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
9,False,False,True,False,False,False,False,False,True,True,...,False,True,False,False,False,True,False,False,False,False


In [121]:
from mlxtend.frequent_patterns import apriori

res=apriori(df, min_support=0.3,use_colnames=True)
res["itemsets_len"]=res["itemsets"].apply(lambda x:len(x))
result=res[res["itemsets_len"]==2]

In [122]:
for i,freq_set in enumerate(result["itemsets"].values):
    print("the {}th freq set is:{}".format(i,list(freq_set)))

the 0th freq set is:['cpu.idle disk.io.util', 'cpu.idle load.average_cpu.5min']
the 1th freq set is:['df.bytes.used.percent disk.io.util', 'cpu.idle load.average_cpu.5min']
the 2th freq set is:['disk.io.util cpu.idle', 'cpu.idle load.average_cpu.5min']
the 3th freq set is:['disk.io.util net.if.out.usage', 'cpu.idle load.average_cpu.5min']
the 4th freq set is:['disk.io.util cpu.idle', 'df.bytes.used.percent disk.io.util']
